In [3]:
### Cargamos dataset One Hot AP5:
import pandas as pd
# Cargar el set de datos total:
dataset = pd.read_csv('H:/Mi unidad/00_Magister/01_Tesis/Control_PID_Code_2/data/OneHot/datasetAP5_OneHot.csv')

## MLP

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np

In [5]:
dataset.head(3)

,Kp,Ti,BAJADA,LINEA RECTA,SUBIDA,LENTA,MEDIA,RAPIDA,CD0,CD2,...,Col_417,Col_418,Col_419,Col_420,Col_421,Col_422,Col_423,Col_424,Col_425,Col_426
0,2.200000,1.172903,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-0.612403,-0.564433,0.480947,0.445399,0.411687,-0.620285,-0.572082,0.473538,0.438202,0.404717
1,3.000000,1.279531,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-0.611063,-0.558706,0.490529,0.451098,0.413925,-0.621107,-0.568424,0.481131,0.442064,0.405216
2,0.633333,1.350616,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,-0.612104,-0.556708,0.495230,0.453288,0.413872,-0.623139,-0.567428,0.484784,0.443170,0.404046


In [6]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Kp,75000.0,0.898333,0.845117,0.100000,0.350000,0.616667,1.100000,3.800000
Ti,75000.0,1.087601,0.253926,0.639765,0.995191,1.172903,1.279531,1.350616
BAJADA,75000.0,0.233333,0.422955,0.000000,0.000000,0.000000,0.000000,1.000000
LINEA RECTA,75000.0,0.033333,0.179507,0.000000,0.000000,0.000000,0.000000,1.000000
SUBIDA,75000.0,0.366667,0.481898,0.000000,0.000000,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...
Col_422,75000.0,-0.618939,0.003966,-0.623830,-0.621898,-0.620345,-0.616778,-0.610151
Col_423,75000.0,-0.574287,0.006963,-0.588637,-0.577986,-0.572205,-0.568424,-0.565392
Col_424,75000.0,0.468142,0.016336,0.437011,0.460996,0.473342,0.481131,0.486969
Col_425,75000.0,0.435588,0.007788,0.419357,0.431650,0.437924,0.442007,0.445557


## División set de datos:

In [7]:
from ai4water.utils.utils import TrainTestSplit

splitter = TrainTestSplit(seed=42,
                          test_fraction=0.3)
TrainX, TestX,_,_ = splitter.split_by_random(dataset)
train_set, val_set, _, _ = splitter.split_by_random(TrainX)

print(f'Tamaño del dataset original: {dataset.shape}\n')
print('Valores de los set antes de la partición del set de entrenamiento:')
print(f'\tSet de entrenamiento: {TrainX.shape}')
print(f'\tTipo: {type(TrainX)}')
print(f'\tSet de prueba: {TestX.shape}')
print(f'\tTipo: {type(TestX)}\n')
print('Valores de los set después de dividir el set de entrenamiento (validation)')
#Todos estos set de datos tienen incluidas las etiquetas.
print(f'\tSet de entrenamiento: {train_set.shape}')
print(f'\tTipo: {type(train_set)}')
# print(f'\tSet de prueba: {TestX.shape}')
print(f'\tSet de validación: {val_set.shape}')
print(f'\tTipo: {type(val_set)}')

c:\Users\Ignac\anaconda3\envs\Tab\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "
c:\Users\Ignac\anaconda3\envs\Tab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tamaño del dataset original: (75000, 440)

Valores de los set antes de la partición del set de entrenamiento:
	Set de entrenamiento: (52500, 440)
	Tipo: <class 'pandas.core.frame.DataFrame'>
	Set de prueba: (22500, 440)
	Tipo: <class 'pandas.core.frame.DataFrame'>

Valores de los set después de dividir el set de entrenamiento (validation)
	Set de entrenamiento: (36750, 440)
	Tipo: <class 'pandas.core.frame.DataFrame'>
	Set de validación: (15750, 440)
	Tipo: <class 'pandas.core.frame.DataFrame'>


In [8]:
LABEL = dataset.columns.to_list()[0:2]
# CAT_FEATURES = dataset.columns.to_list()[2:10]
NUMERIC_FEATURES = dataset.columns.to_list()[2:440]
dataset[NUMERIC_FEATURES] = dataset[NUMERIC_FEATURES].astype(float)
dataset[LABEL] = dataset[LABEL].astype(float)
# dataset[CAT_FEATURES] = dataset[CAT_FEATURES].astype(float)
print(f'Características numericas: {NUMERIC_FEATURES}\nObjetivo: {LABEL}')#\nCaracterísticas Categóricas: {CAT_FEATURES}

Características numericas: ['BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2', 'Kp_m', 'Ti_m', 'tau_p', 'K', 'Col_1', 'Col_2', 'Col_3', 'Col_4', 'Col_5', 'Col_6', 'Col_7', 'Col_8', 'Col_9', 'Col_10', 'Col_11', 'Col_12', 'Col_13', 'Col_14', 'Col_15', 'Col_16', 'Col_17', 'Col_18', 'Col_19', 'Col_20', 'Col_21', 'Col_22', 'Col_23', 'Col_24', 'Col_25', 'Col_26', 'Col_27', 'Col_28', 'Col_29', 'Col_30', 'Col_31', 'Col_32', 'Col_33', 'Col_34', 'Col_35', 'Col_36', 'Col_37', 'Col_38', 'Col_39', 'Col_40', 'Col_41', 'Col_42', 'Col_43', 'Col_44', 'Col_45', 'Col_46', 'Col_47', 'Col_48', 'Col_49', 'Col_50', 'Col_51', 'Col_52', 'Col_53', 'Col_54', 'Col_55', 'Col_56', 'Col_57', 'Col_58', 'Col_59', 'Col_60', 'Col_61', 'Col_62', 'Col_63', 'Col_64', 'Col_65', 'Col_66', 'Col_67', 'Col_68', 'Col_69', 'Col_70', 'Col_71', 'Col_72', 'Col_73', 'Col_74', 'Col_75', 'Col_76', 'Col_77', 'Col_78', 'Col_79', 'Col_80', 'Col_81', 'Col_82', 'Col_83', 'Col_84', 'Col_85', 'Col_86', 'Col_87', 'Col

In [9]:
ONE_HOT = dataset.columns.to_list()[2:10]
ONE_HOT

['BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2']

In [10]:
# Datos de entrenamiento:
# train_x_all = TrainX[NUMERIC_FEATURES]
# train_y_all = TrainX[LABEL]

# Datos de prueba:
test_x_all = TestX[NUMERIC_FEATURES].values
test_y_all =TestX[LABEL]

# Datos de entrenamiento a utilizar en el fit:
train_x_all = train_set[NUMERIC_FEATURES].values
train_y_all = train_set[LABEL]
# Datos de validación a utilizar en el fit:
val_x_all = val_set[NUMERIC_FEATURES].values
val_y_all = val_set[LABEL]

In [11]:
train_x_all

array([[0.        , 0.        , 1.        , ..., 0.47469264, 0.43923043,
        0.40559974],
       [0.        , 0.        , 0.        , ..., 0.47032114, 0.43505761,
        0.40163311],
       [0.        , 0.        , 1.        , ..., 0.47353795, 0.4382016 ,
        0.40471715],
       ...,
       [1.        , 0.        , 0.        , ..., 0.43902989, 0.42156834,
        0.40398645],
       [0.        , 0.        , 0.        , ..., 0.47032114, 0.43505761,
        0.40163311],
       [0.        , 0.        , 1.        , ..., 0.48601356, 0.4444858 ,
        0.40543654]])

In [12]:
train_y_all

,Kp,Ti
52017,0.733333,1.172903
68437,0.366667,1.172903
22169,2.200000,1.172903
8321,0.700000,0.995191
1926,3.000000,1.279531
...,...,...
46530,0.633333,1.350616
44305,0.366667,1.172903
36464,0.200000,0.639765
55825,0.366667,1.172903


### Para ingresar los datos a la red MLP, necesitamos escalar el set de entrenamiento, de prueba y de validación. *Las caracterpisticas objetivos no se escalan*

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_all_scaled = scaler.fit_transform(train_x_all)
X_test_all_scaled = scaler.fit_transform(test_x_all)
X_val_all_scaled = scaler.fit_transform(val_x_all)

In [104]:
train_x_all[0]

array([ 0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.61020408,  0.86428571,
        2.        ,  3.        ,  0.75535078,  0.73980759,  0.72433107,
        0.70815912, -0.06311351, -0.06336879, -0.06267975, -0.06246239,
        0.69368737,  0.6799163 , -0.08964913, -0.08946863, -0.08779481,
       -0.08679242, -0.08567428, -0.08442884, -0.08204588,  0.67575603,
        0.66303775,  0.65017491,  0.63702127,  0.62422254,  0.61095347,
       -0.15675939, -0.15424565,  0.60457311,  0.59305965,  0.58088165,
        0.56937425, -0.19764964, -0.1940004 , -0.18968754, -0.18535941,
       -0.17996851, -0.17455111, -0.16906099, -0.16316845, -0.15714844,
        0.60438464,  0.59450561,  0.58568091,  0.57668803, -0.18787865,
       -0.18211171, -0.1764149 ,  0.58539513,  0.5756387 ,  0.56679967,
        0.55767102,  0.54892294,  0.53975188, -0.22450765, -0.21868979,
       -0.21245802, -0.20543248, -0.19790772, -0.19128896, -0.18

##### Datos de entrenamiento escalado:

In [14]:
print(f'Datos de entrenamiento escalados sin LABEL:\n\tTipo de dato: {type(X_train_all_scaled)}\n\tNumero de filas: {len(X_train_all_scaled)}\n\tNumero de datos en una fila: {len(X_train_all_scaled[0])}')
print(f'Cómo van los datos en una fila:\n{X_train_all_scaled[0]}')

Datos de entrenamiento escalados sin LABEL:
	Tipo de dato: <class 'numpy.ndarray'>
	Numero de filas: 36750
	Numero de datos en una fila: 438
Cómo van los datos en una fila:
[-0.55155141 -0.18545999  1.30873135  0.81312183 -0.50021258 -0.49668144
 -0.26338354 -0.65465367 -0.31068084 -0.24379496 -0.00519804 -0.29418827
 -0.19928618 -0.20071092 -0.20148997 -0.20329791  0.14861993  0.14627932
  0.15008141  0.14747095 -0.20499997 -0.2051057   0.15655979  0.15234362
  0.15394867  0.1501629   0.14577172  0.14133967  0.14186147 -0.20762665
 -0.20820083 -0.20895117 -0.21030962 -0.2110731  -0.21278425  0.15276883
  0.15129723 -0.2135124  -0.21389751 -0.21574816 -0.21667628  0.15204316
  0.149638    0.14821766  0.1456408   0.14476207  0.14393074  0.14217803
  0.14156091  0.14033748 -0.21739735 -0.21930445 -0.21930244 -0.21918712
  0.14900336  0.14843111  0.14726589 -0.21727405 -0.21910495 -0.21946342
 -0.22003965 -0.22003625 -0.2207837   0.15351775  0.15196225  0.15056908
  0.15060143  0.15146966

##### Datos de validación escalado:

In [15]:
print(f'Datos de entrenamiento escalados sin LABEL:\n\tTipo de dato: {type(X_val_all_scaled)}\n\tNumero de filas: {len(X_val_all_scaled)}\n\tNumero de datos en una fila: {len(X_val_all_scaled[0])}')
print(f'Cómo van los datos en una fila:\n{X_val_all_scaled[0]}')

Datos de entrenamiento escalados sin LABEL:
	Tipo de dato: <class 'numpy.ndarray'>
	Numero de filas: 15750
	Numero de datos en una fila: 438
Cómo van los datos en una fila:
[-0.55588606 -0.18237645  1.31894158  0.81919932 -0.49662467 -0.50584653
 -0.26957202 -0.65307061 -0.65566806  1.6885448   0.00700145 -0.87741297
  0.23935621  0.24663408  0.2535368   0.26090018 -0.07318152 -0.0885842
 -0.09782618 -0.1087161   0.25600631  0.26296616 -0.10037989 -0.10759081
 -0.11431723 -0.12292691 -0.13273388 -0.14238152 -0.14738112  0.25165266
  0.25946404  0.26592967  0.27320885  0.27925816  0.28572374 -0.11420454
 -0.12289663  0.28013512  0.28474451  0.29185906  0.29787768 -0.12156717
 -0.12996256 -0.13830922 -0.14833936 -0.15726344 -0.16587252 -0.17252863
 -0.17951555 -0.18682595  0.25207491  0.25965966  0.26661202  0.27375324
 -0.15010724 -0.15820139 -0.16564513  0.26319654  0.26926869  0.27550225
  0.28118999  0.28567592  0.29301791 -0.14276847 -0.14934755 -0.15707013
 -0.16400495 -0.1695848  

##### Datos de prueba escalados:

In [16]:
print(f'Datos de validación escalados sin LABEL:\n\tTipo de dato: {type(X_test_all_scaled)}\n\tNumero de filas: {len(X_test_all_scaled)}\n\tNumero de datos en una fila: {len(X_test_all_scaled[0])}')
print(f'Cómo van los datos en una fila:\n{X_test_all_scaled[0]}')

Datos de validación escalados sin LABEL:
	Tipo de dato: <class 'numpy.ndarray'>
	Numero de filas: 22500
	Numero de datos en una fila: 438
Cómo van los datos en una fila:
[-0.54893556 -0.18836808  1.32006235  0.82013032 -0.50201309 -0.5013191
 -0.27190884 -0.65576227 -0.66168975  0.79844426  1.41686106 -0.28994626
  0.44311599  0.42375499  0.40282325  0.38168966 -0.81706206 -0.80413928
 -0.79009869 -0.77891464  0.38395596  0.36370023 -0.77536427 -0.7677132
 -0.75950522 -0.75431242 -0.74434343 -0.73619139 -0.72997162  0.38744122
  0.36651617  0.34626948  0.32434562  0.30397658  0.2811801  -0.75394444
 -0.7466905   0.29174664  0.27006178  0.24741243  0.22529472 -0.74303472
 -0.7332444  -0.72383995 -0.71263647 -0.70147668 -0.69306551 -0.68220684
 -0.67650789 -0.66487839  0.3582208   0.33811063  0.3180622   0.29592379
 -0.70413138 -0.69330626 -0.68060078  0.32685514  0.30588824  0.28408282
  0.26122655  0.23892369  0.21514606 -0.71645058 -0.70477153 -0.69338566
 -0.68412835 -0.67355243 -0.6

#### Modelo MLP:

In [17]:
def MultiLayerPerceptron(input_dim):
    # Regularización L2
    l2_reg = l2(0.00000001)
    # Capa de entrada
    inputs = Input(shape=(input_dim,))

    # Capas ocultas:
    x = Dense(438, activation='linear', kernel_regularizer=l2_reg)(inputs)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='linear', kernel_regularizer=l2_reg)(x)
    x = Dropout(0.2)(x)
    x = Dense(63, activation='linear' ,kernel_regularizer=l2_reg)(x)

    # Output de las variables Kp y Ti
    output_Kp = Dense(1, activation='linear', name='Ganancia_Kp')(x)
    output_Ti = Dense(1, activation='linear', name='Periodo_Ti')(x)

    # Modelo
    model = Model(inputs=inputs, outputs = [output_Kp, output_Ti])
    return model

##### Mejor modelo hasta el momento: 12/12/2024

##### Modelo_05

In [16]:
def MultiLayerPerceptron(input_dim):
    # Regularización L2
    l2_reg = l2(0.000001)
    # Capa de entrada
    inputs = Input(shape=(input_dim,))

    # Capas ocultas:
    x = Dense(438, activation='linear', kernel_regularizer=l2_reg)(inputs)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='linear', kernel_regularizer=l2_reg)(x)
    x = Dropout(0.2)(x)
    x = Dense(63, activation='linear' ,kernel_regularizer=l2_reg)(x)

    # Output de las variables Kp y Ti
    output_Kp = Dense(1, activation='linear', name='Ganancia_Kp')(x)
    output_Ti = Dense(1, activation='linear', name='Periodo_Ti')(x)

    # Modelo
    model = Model(inputs=inputs, outputs = [output_Kp, output_Ti])
    return model

##### Modelo_04


In [ ]:
def MultiLayerPerceptron(input_dim):
    # Regularización L2
    l2_reg = l2(0.000002)
    # Capa de entrada
    inputs = Input(shape=(input_dim,))

    # Capas ocultas:
    x = Dense(438, activation='linear', kernel_regularizer=l2_reg)(inputs)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='linear', kernel_regularizer=l2_reg)(x)
    x = Dropout(0.2)(x)
    x = Dense(63, activation='linear' ,kernel_regularizer=l2_reg)(x)

    # Output de las variables Kp y Ti
    output_Kp = Dense(1, activation='linear', name='Ganancia_Kp')(x)
    output_Ti = Dense(1, activation='linear', name='Periodo_Ti')(x)

    # Modelo
    model = Model(inputs=inputs, outputs = [output_Kp, output_Ti])
    return model

##### Modelo_03

In [ ]:
def MultiLayerPerceptron(input_dim):
    # Regularización L2
    l2_reg = l2(0.000003)
    # Capa de entrada
    inputs = Input(shape=(input_dim,))

    # Capas ocultas:
    x = Dense(438, activation='linear', kernel_regularizer=l2_reg)(inputs)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='linear', kernel_regularizer=l2_reg)(x)
    x = Dropout(0.2)(x)
    x = Dense(63, activation='linear' ,kernel_regularizer=l2_reg)(x)

    # Output de las variables Kp y Ti
    output_Kp = Dense(1, activation='linear', name='Ganancia_Kp')(x)
    output_Ti = Dense(1, activation='linear', name='Periodo_Ti')(x)

    # Modelo
    model = Model(inputs=inputs, outputs = [output_Kp, output_Ti])
    return model

##### Modelo 1

In [ ]:
def MultiLayerPerceptron(input_dim):
    # Regularización L2
    l2_reg = l2(0.000003)
    # Capa de entrada
    inputs = Input(shape=(input_dim,))

    # Capas ocultas:
    x = Dense(438, activation='linear', kernel_regularizer=l2_reg)(inputs)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='linear', kernel_regularizer=l2_reg)(x)
    x = Dropout(0.2)(x)
    x = Dense(63, activation='linear' ,kernel_regularizer=l2_reg)(x)

    # Output de las variables Kp y Ti
    output_Kp = Dense(1, activation='linear', name='Ganancia_Kp')(x)
    output_Ti = Dense(1, activation='linear', name='Periodo_Ti')(x)

    # Modelo
    model = Model(inputs=inputs, outputs = [output_Kp, output_Ti])
    return model

##### Modelo 0

In [ ]:
def MultiLayerPerceptron(input_dim):
    # Regularización L2
    l2_reg = l2(0.0003)
    # Capa de entrada
    inputs = Input(shape=(input_dim,))

    # Capas ocultas:
    x = Dense(128, activation='linear', kernel_regularizer=l2_reg)(inputs)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='linear', kernel_regularizer=l2_reg)(x)
    x = Dropout(0.2)(x)
    x = Dense(31, activation='linear' ,kernel_regularizer=l2_reg)(x)

    # Output de las variables Kp y Ti
    output_Kp = Dense(1, activation='linear', name='Ganancia_Kp')(x)
    output_Ti = Dense(1, activation='linear', name='Periodo_Ti')(x)

    # Modelo
    model = Model(inputs=inputs, outputs = [output_Kp, output_Ti])
    return model

#### Creamos el modelo:

In [18]:
input_dim = X_train_all_scaled.shape[1]
mlp_model = MultiLayerPerceptron(input_dim=input_dim)

In [19]:
input_dim

438

#### Compilamos el modelo:

In [20]:
from tensorflow.keras.optimizers import Adam
mlp_model.compile(optimizer=Adam(learning_rate=0.005),
                  loss={'Ganancia_Kp': 'mse', 'Periodo_Ti': 'mse'},
                  metrics={'Ganancia_Kp': ['mse','mae'], 'Periodo_Ti': ['mse','mae']})

#### Resumen del modelo:

In [21]:
mlp_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 438)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 438)          192282      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 438)          0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          56192       dropout[0][0]                    
______________________________________________________________________________________________

### Early Stopping:

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_Ganancia_Kp_loss',
                               patience=10,
                               restore_best_weights=True)

### Reduce LR On Plateau:

In [23]:
lr_scheduler = ReduceLROnPlateau(monitor='val_Ganancia_Kp_loss',
                                 factor=0.2,
                                 patience=5,
                                 verbose=0)

#### Entrenamiento del modelo:

##### fit código con valores escalados en el MLP ai4water:

In [24]:
history = mlp_model.fit(x= X_train_all_scaled,
          y={'Ganancia_Kp': train_y_all['Kp'], 'Periodo_Ti': train_y_all['Ti']},
          validation_data = (X_val_all_scaled,
                             {'Ganancia_Kp': val_y_all['Kp'], 'Periodo_Ti': val_y_all['Ti']}),
          callbacks = [early_stopping, lr_scheduler],
          epochs=500,
          verbose=1)

Epoch 1/500
1149/1149 [==============================] - 6s 5ms/step - loss: 12.4661 - Ganancia_Kp_loss: 2.8531 - Periodo_Ti_loss: 9.6130 - Ganancia_Kp_mse: 2.8531 - Ganancia_Kp_mae: 0.3603 - Periodo_Ti_mse: 9.6130 - Periodo_Ti_mae: 0.4031 - val_loss: 0.0047 - val_Ganancia_Kp_loss: 0.0012 - val_Periodo_Ti_loss: 0.0035 - val_Ganancia_Kp_mse: 0.0012 - val_Ganancia_Kp_mae: 0.0248 - val_Periodo_Ti_mse: 0.0035 - val_Periodo_Ti_mae: 0.0384
Epoch 2/500
1149/1149 [==============================] - 5s 4ms/step - loss: 0.0262 - Ganancia_Kp_loss: 0.0203 - Periodo_Ti_loss: 0.0059 - Ganancia_Kp_mse: 0.0203 - Ganancia_Kp_mae: 0.0963 - Periodo_Ti_mse: 0.0059 - Periodo_Ti_mae: 0.0509 - val_loss: 0.0405 - val_Ganancia_Kp_loss: 0.0401 - val_Periodo_Ti_loss: 4.3400e-04 - val_Ganancia_Kp_mse: 0.0401 - val_Ganancia_Kp_mae: 0.1496 - val_Periodo_Ti_mse: 4.3400e-04 - val_Periodo_Ti_mae: 0.0158
Epoch 3/500
1149/1149 [==============================] - 5s 5ms/step - loss: 0.0422 - Ganancia_Kp_loss: 0.0306 - Peri

In [28]:
save_dir = f"MLP_01"

#### Guardamos datos del History:

In [29]:
import os
import matplotlib.pyplot as plt

In [30]:
hist = pd.DataFrame(history.history)
hist.to_csv(os.path.join(save_dir, "MLP.csv"), index=False)

#### Graficas:

In [31]:
def save_loss_plots(history, save_dir):
    # Pérdida total
    plt.figure()
    plt.plot(history.history['loss'], label='Loss (Train)', color='red')
    plt.plot(history.history['val_loss'], label='Loss (Validation)', color='orange', linestyle='--')
    plt.title('Total Loss', fontsize=14)
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, "total_loss.png"), dpi=300, bbox_inches='tight')
    plt.close()

    # Pérdida de Kp
    plt.figure()
    plt.plot(history.history['Ganancia_Kp_loss'], label='Kp Loss (Train)', color='blue')
    plt.plot(history.history['val_Ganancia_Kp_loss'], label='Kp Loss (Validation)', color='cyan', linestyle='--')
    plt.title('Kp Loss', fontsize=14)
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, "kp_loss.png"), dpi=300, bbox_inches='tight')
    plt.close()

    # Pérdida de Ti
    plt.figure()
    plt.plot(history.history['Periodo_Ti_loss'], label='Ti Loss (Train)', color='green')
    plt.plot(history.history['val_Periodo_Ti_loss'], label='Ti Loss (Validation)', color='lime', linestyle='--')
    plt.title('Ti Loss', fontsize=14)
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, "ti_loss.png"), dpi=300, bbox_inches='tight')
    plt.close()

In [32]:
save_loss_plots(history, save_dir)

#### Evaluamos el modelo en el conjunto de prueba:

In [33]:
results = mlp_model.evaluate(
    X_test_all_scaled,
    {'Ganancia_Kp': test_y_all['Kp'], 'Periodo_Ti': test_y_all['Ti']}
)
print(f'Resultados en el conjunto de prueba: {results}')

704/704 [==============================] - 2s 3ms/step - loss: 2.4203e-04 - Ganancia_Kp_loss: 2.1366e-04 - Periodo_Ti_loss: 2.1702e-05 - Ganancia_Kp_mse: 2.1366e-04 - Ganancia_Kp_mae: 0.0096 - Periodo_Ti_mse: 2.1702e-05 - Periodo_Ti_mae: 0.0035
Resultados en el conjunto de prueba: [0.0002420282835373655, 0.00021365639986470342, 2.1702282538171858e-05, 0.00021365639986470342, 0.009632495231926441, 2.1702282538171858e-05, 0.0034948785323649645]


#### Prediccion en el conjunto de test:

In [34]:
y_pred = mlp_model.predict(X_test_all_scaled)
r2_kp = r2_score(test_y_all['Kp'], y_pred[0])
r2_ti = r2_score(test_y_all['Ti'], y_pred[1])
print(f"R² para Kp: {r2_kp:.4f}")
print(f"R² para Ti: {r2_ti:.4f}")

R² para Kp: 0.9997
R² para Ti: 0.9997


In [36]:
bloque = tf.keras.utils.plot_model(mlp_model,
                                   to_file=os.path.join(save_dir, "MLP.png"),
                          rankdir='TB')

In [35]:
hist = pd.DataFrame(history.history)
hist

,loss,Ganancia_Kp_loss,Periodo_Ti_loss,Ganancia_Kp_mse,Ganancia_Kp_mae,Periodo_Ti_mse,Periodo_Ti_mae,val_loss,val_Ganancia_Kp_loss,val_Periodo_Ti_loss,val_Ganancia_Kp_mse,val_Ganancia_Kp_mae,val_Periodo_Ti_mse,val_Periodo_Ti_mae,lr
0,12.466095,2.853133,9.612963,2.853133,0.360258,9.612963,0.403130,0.004672,0.001194,0.003471,0.001194,0.024801,0.003471,0.038378,5.000000e-03
1,0.026200,0.020273,0.005919,0.020273,0.096304,0.005919,0.050878,0.040524,0.040082,0.000434,0.040082,0.149556,0.000434,0.015778,5.000000e-03
2,0.042184,0.030597,0.011580,0.030597,0.116560,0.011580,0.070541,0.008858,0.008398,0.000452,0.008398,0.063228,0.000452,0.015918,5.000000e-03
3,0.036980,0.027108,0.009864,0.027108,0.109259,0.009864,0.065875,0.003335,0.002700,0.000628,0.002700,0.042042,0.000628,0.021101,5.000000e-03
4,0.024073,0.017544,0.006522,0.017544,0.087867,0.006522,0.054129,0.011066,0.010837,0.000222,0.010837,0.074574,0.000222,0.012381,5.000000e-03
5,0.015568,0.011363,0.004198,0.011363,0.071208,0.004198,0.043466,0.019407,0.016021,0.003379,0.016021,0.102300,0.003379,0.047016,5.000000e-03
6,0.007405,0.005941,0.001456,0.005941,0.050629,0.001456,0.026413,0.001934,0.001548,0.000379,0.001548,0.029580,0.000379,0.011723,9.999999e-04
7,0.006982,0.005555,0.001420,0.005555,0.048879,0.001420,0.026375,0.001066,0.000944,0.000115,0.000944,0.023347,0.000115,0.008116,9.999999e-04
8,0.008632,0.006658,0.001967,0.006658,0.053456,0.001967,0.030760,0.004316,0.003915,0.000395,0.003915,0.048073,0.000395,0.013007,9.999999e-04
9,0.014586,0.010931,0.003648,0.010931,0.069169,0.003648,0.040829,0.003610,0.003295,0.000307,0.003295,0.043688,0.000307,0.013031,9.999999e-04


#### Guardar los pesos del modelo:

In [ ]:
# Guardar los pesos del modelo
mlp_model.save_weights('mlp_model_weights_06.h5')
print("Pesos del modelo guardados como 'mlp_model_weights_06.h5'")

Pesos del modelo guardados como 'mlp_model_weights_06.h5'


### Modelo completo:

In [37]:
# Guardar el modelo completo
import tensorflow as tf
# model.save('Modelo_Basico_01/Modelo_01.h5')
mlp_model.save(os.path.join(save_dir, "Modelo_MLP.h5"))
print("Modelo completo guardado como 'Modelo_MLP.h5'")

Modelo completo guardado como 'Modelo_MLP.h5'


### Cargar el modelo:

In [2]:
from tensorflow.keras.models import load_model
# Cargar el modelo completo
loaded_model = load_model('H:/Mi unidad/00_Magister/01_Tesis/Control_PID_Code_2/MLP_TensorFlow/MLP_01/Modelo_MLP.h5')
print("Modelo completo cargado desde 'Modelo_MLP.h5'")

Modelo completo cargado desde 'Modelo_MLP.h5'


In [17]:
results = loaded_model.evaluate(
    X_test_all_scaled,
    {'Ganancia_Kp': test_y_all['Kp'], 'Periodo_Ti': test_y_all['Ti']}
)
print(f'Resultados en el conjunto de prueba: {results}')

704/704 [==============================] - 2s 3ms/step - loss: 2.4203e-04 - Ganancia_Kp_loss: 2.1366e-04 - Periodo_Ti_loss: 2.1702e-05 - Ganancia_Kp_mse: 2.1366e-04 - Ganancia_Kp_mae: 0.0096 - Periodo_Ti_mse: 2.1702e-05 - Periodo_Ti_mae: 0.0035
Resultados en el conjunto de prueba: [0.0002420282835373655, 0.00021365639986470342, 2.1702282538171858e-05, 0.00021365639986470342, 0.009632495231926441, 2.1702282538171858e-05, 0.0034948785323649645]


In [18]:
y_pred = loaded_model.predict(X_test_all_scaled)
r2_kp = r2_score(test_y_all['Kp'], y_pred[0])
r2_ti = r2_score(test_y_all['Ti'], y_pred[1])
print(f"R² para Kp: {r2_kp:.4f}")
print(f"R² para Ti: {r2_ti:.4f}")

R² para Kp: 0.9997
R² para Ti: 0.9997


In [100]:
y_pred[1][0]

array([1.3492025], dtype=float32)

In [93]:
y_pred[0][0][0]

1.281405

In [101]:
y_pred[1][0][0]

1.3492025

In [102]:
print(f'Kp: {y_pred[0][0]}')
print(f'Ti: {y_pred[1][0]}')

Kp: [1.281405]
Ti: [1.3492025]


### Análisis de como entran los datos al simulador:

In [25]:
print('Total de filas:', len(X_test_all_scaled))
print(f'Una sola fila: {len(X_test_all_scaled[0])}')

Total de filas: 22500
Una sola fila: 438


In [44]:
len(test_x_all.values[0])

438

In [34]:
test_x_all = TrainX[NUMERIC_FEATURES]
test_x_all.columns.to_list()[0:8]

['BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2']

In [68]:
test_x_all

,BAJADA,LINEA RECTA,SUBIDA,LENTA,MEDIA,RAPIDA,CD0,CD2,Kp_m,Ti_m,...,Col_417,Col_418,Col_419,Col_420,Col_421,Col_422,Col_423,Col_424,Col_425,Col_426
26837,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.724490,0.604082,...,-0.612202,-0.556888,0.494978,0.453064,0.413627,-0.623449,-0.567703,0.484633,0.443137,0.404155
2592,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.006122,0.864286,...,-0.612048,-0.564152,0.481127,0.445455,0.411667,-0.620406,-0.572328,0.473147,0.437647,0.404013
18359,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.175510,1.297959,...,-0.612569,-0.571840,0.467098,0.437713,0.409572,-0.617388,-0.576748,0.462126,0.432807,0.404724
73292,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.855102,0.846939,...,-0.614843,-0.574074,0.464899,0.435638,0.407514,-0.619430,-0.578685,0.460283,0.430955,0.402773
60127,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.704082,0.569388,...,-0.610954,-0.562970,0.482423,0.446885,0.413114,-0.618906,-0.570785,0.474693,0.439230,0.405600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15446,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.800000,1.124490,...,-0.611063,-0.558706,0.490529,0.451098,0.413925,-0.621107,-0.568424,0.481131,0.442064,0.405216
2512,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.346939,1.228571,...,-0.613609,-0.561236,0.487987,0.448627,0.411574,-0.623270,-0.570365,0.479498,0.440704,0.403988
37620,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.704082,1.176531,...,-0.610954,-0.562970,0.482423,0.446885,0.413114,-0.618906,-0.570785,0.474693,0.439230,0.405600
38764,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.422449,1.315306,...,-0.612048,-0.564152,0.481127,0.445455,0.411667,-0.620406,-0.572328,0.473147,0.437647,0.404013


In [35]:
cat = ['BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2']
cat

['BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2']

In [47]:
import control as ctr
import TabSynth as TSS
def asignacion_categoria(Kp, Ti, K, tau_p, T=0.07043):
    num_cd = [Kp*(1+T/(2*Ti)), -Kp*(1-T/(2*Ti))]
    den_cd = [1, -1]
    Cd = ctr.TransferFunction(num_cd,den_cd, T)
    e = np.exp(-T/tau_p)
    num_gd = K*(1 -e)
    den_gd = [1, -e]
    Gd = ctr.TransferFunction(num_gd,den_gd, T)
    #Lazo cerrado:
    H = ctr.feedback(Cd*Gd,1)
    info = ctr.step_info(H)
    Valor_ess = info['SteadyStateValue']# Valor estado estable
    Tiempo_estable = info['SettlingTime']# Tiempo de asentamiento
    Overshoot = info['Overshoot']
    categoria = TSS.Categoria(K, tau_p, K_nominal=3.0, tau_nominal=1.5)
    cat_1 = categoria.categoria_pendiente()
    cat_2 = categoria.categoria_(Tiempo_estable)
    return [cat_1, cat_2]

In [48]:
dataset[['Kp','Ti','Kp_m', 'Ti_m', 'tau_p', 'K', 'BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2']].iloc[0:2]

,Kp,Ti,Kp_m,Ti_m,tau_p,K,BAJADA,LINEA RECTA,SUBIDA,LENTA,MEDIA,RAPIDA,CD0,CD2
0,2.2,1.172903,3.271429,1.297959,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,3.0,1.279531,1.685714,0.760204,2.5,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [49]:
output = asignacion_categoria(Kp=2.2, Ti=1.172903, K=10, tau_p=2.0)

In [50]:
print(type(output))

<class 'list'>


In [51]:
output

['CD2', 'RAPIDA']

In [52]:
cat

['BAJADA', 'LINEA RECTA', 'SUBIDA', 'LENTA', 'MEDIA', 'RAPIDA', 'CD0', 'CD2']

In [53]:
indicadores = [0.0]*len(cat)
indicadores

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [54]:
for i, categoria in enumerate(cat):
    if categoria in output:
        indicadores[i] = 1.0

In [55]:
indicadores

[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

#### Función para las categorias:

In [56]:
def valor_cat(cat, output):
    # Creamos lista de ceros, del mismo tamaño que las categorias:
    indicadores = [0.0]*len(cat)
    for i, categoria in enumerate(cat):
        if categoria in output:
            indicadores[i] = 1.0
    return indicadores

In [57]:
ind = valor_cat(cat, output)
ind

[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]

In [58]:
cero_tres = [2.2, 4.3, 2.0, 10]
union = cero_tres + ind
print(len(union))
print(union)

12
[2.2, 4.3, 2.0, 10, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]


In [59]:
union_array = np.array(union).reshape(1,-1)
union_array

array([[ 2.2,  4.3,  2. , 10. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,
         1. ]])

In [60]:
union_scale = scaler.fit_transform(union_array)
union_scale

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [61]:
union_scale[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [63]:
X_test_all_scaled[0]

array([-0.54893556, -0.18836808,  1.32006235,  0.82013032, -0.50201309,
       -0.5013191 , -0.27190884, -0.65576227, -0.66168975,  0.79844426,
        1.41686106, -0.28994626,  0.44311599,  0.42375499,  0.40282325,
        0.38168966, -0.81706206, -0.80413928, -0.79009869, -0.77891464,
        0.38395596,  0.36370023, -0.77536427, -0.7677132 , -0.75950522,
       -0.75431242, -0.74434343, -0.73619139, -0.72997162,  0.38744122,
        0.36651617,  0.34626948,  0.32434562,  0.30397658,  0.2811801 ,
       -0.75394444, -0.7466905 ,  0.29174664,  0.27006178,  0.24741243,
        0.22529472, -0.74303472, -0.7332444 , -0.72383995, -0.71263647,
       -0.70147668, -0.69306551, -0.68220684, -0.67650789, -0.66487839,
        0.3582208 ,  0.33811063,  0.3180622 ,  0.29592379, -0.70413138,
       -0.69330626, -0.68060078,  0.32685514,  0.30588824,  0.28408282,
        0.26122655,  0.23892369,  0.21514606, -0.71645058, -0.70477153,
       -0.69338566, -0.68412835, -0.67355243, -0.66414833, -0.65

---

# Valores obtenidos de la simulación:

In [157]:
import numpy as np
array_simulacion = np.array([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.95168346, 1.1343101, 9, 12, 1.1786419565754653, 1.1378605933581105, 1.0957252274435885, 1.0525957592980566, 1.0085274455409556, 0.9644921785357082, 0.9186058438444681, 0.8738461223809529, 0.8291499362589614, 0.7853971564259835, 0.7416208104599047, 0.6996321879111795, 0.6583101729943153, 0.6174823781330897, 0.5780708368961676, 0.5409135623756179, 0.5048507757287841, 0.4705974145254612, 0.4380373899881282, 0.4069110566402095, 0.3771397410724, 0.3484974873787485, 0.321780233258548, 0.2969454310587614, 0.27364453650817283, 0.25271822295169255, 0.23300910689328666, 0.21431214456079278, 0.19651996520930204, 0.1814580281631379, 0.1669407214897759, 0.1539072301102218, 0.14213314871012922, 0.1314936730119578, 0.12187642666547954, 0.11418022368300756, 0.10722216978830611, 0.10092936751825439, 0.09615831665239219, 0.09184313904763508, 0.08886118424231654, 0.0861653476469717, 0.08364988289551509, 0.08229682867917253, 0.08099822794220445, 0.08074865824319317, 0.08145145278164719, 0.08201957077305651, 0.08346642321879437, 0.08471230133939842, 0.08677721802508274, 0.08950954713505968, 0.09200573994032937, 0.09413419399510149, 0.09693092154639112, 0.09949228812019786, 0.10168661406415823, 0.10462770394534243, 0.1081711286825465, 0.11134195808947839, 0.1141772949623984, 0.11671054742568412, 0.11989393500636325, 0.12274776759976978, 0.1262270002348628, 0.12935440590943564, 0.13216501931924068, 0.1346903864282278, 0.13788105003569856, 0.1407558102766979, 0.14334612459927124, 0.14560245610117226, 0.14863531747472403, 0.15137517740512843, 0.1528512127820721, 0.15510361756363958, 0.15714013131843366, 0.15898225233194005, 0.16064933810461118, 0.16215881852982322, 0.16344852600526508, 0.16369546211839056, 0.16484055359268873, 0.1658794126318548, 0.16682261779143326, 0.1676796942214123, 0.16845921856365215, 0.169091051565249, 0.16974501697475164, 0.17034165595743617, 0.1708088152024778, 0.17131450495905778, 0.17169977164170455, 0.17213172364302454, 0.17245059532725196, 0.1718227639001293, 0.17217958967232996, 0.17250807105178645, 0.1728888923928053, 0.173161724974026, 0.17249243898728756, 0.17181193821746796, 0.17219859432944093, 0.17163179740163484, 0.17204357989812652, 0.17149940728610716, 0.17093156118400105, 0.17049751572492183, 0.1700288303867912, 0.1696840713284738, 0.17029578131044126, 0.169931627537284, 0.16960373595571912, 0.17023063376227388, 0.1698801554201884, 0.16956457747932507, 0.17020256274239395, 0.16986206777524535, 0.16955547907145338, 0.17020155842931867, 0.16986835155236615, 0.16956832519557477, 0.17022031342627197, 0.16989242702050794, 0.16959719132590684, 0.16933135500627258, 0.16901412865286672, 0.1688057463938697, 0.16861811470223526, 0.16844916725020628, 0.16821918163512878, 0.16808935273950673, 0.16797245211933987, 0.16786719238551867, 0.16777241434628973, 0.1676870742414464, 0.16761023224773175, 0.1675410421288674, 0.1674787419162226, 0.16642264551749886, 0.1654639194832128, 0.1645928677369515, 1.0880507561635366, 1.186005076601796, 1.2792544915617579, 1.3677426873461536, 1.451418957938398, 1.5303239035698453, 1.6043215675749274, 1.6735555859029125, 1.738069034353971, 1.7979869562450004, 1.8533429693967747, 1.9043398579425053, 1.9510739559293153, 1.993624721312752, 2.032157239865105, 2.066913698414115, 2.0980259100072076, 2.125691535601259, 2.150095836416904, 2.171398342274149, 2.189749980330119, 2.205279365304618, 2.218188561525206, 2.228674078309417, 2.2368982902851626, 2.2431009431097695, 2.2474116517503786, 2.249939840990579, 2.2507840391664686, 2.2502127603914066, 2.2482879831574256, 2.2451590543667037, 2.2409531658448394, 2.235784843739664, 2.2297572097610465, 2.223056657014611, 2.215757477516613, 2.2079265653335236, 2.1997104102413565, 2.1911546576848484, 2.1823866647419603, 2.17343484938078, 2.1643175165159954, 2.155144688458351, 2.145921891382404, 2.1367476413572466, 2.127710644287048, 2.118797232613441, 2.1100886399718095, 2.1015644706185777, 2.0932998999921493, 2.0853553343995976, 2.077706191162507, 2.0703157691073923, 2.0632445619266577, 2.056468065625449, 2.049949649878582, 2.043757155382692, 2.0379443840918134, 2.0324735234206397, 2.0273105261360698, 2.0224247354059295, 2.0178748041614765, 2.0136272877375276, 2.009738328842898, 2.0061722225562506, 2.002896819428672, 1.9998831714320822, 1.9971914701402536, 1.9947896563135155, 1.992648863150054, 1.9907358151741106, 1.9891213738580618, 1.987775804148449, 1.9865787918014082, 1.9856018005495317, 1.9848229208378585, 1.9842224248504026, 1.98378254925403, 1.9834872995765576, 1.9833149908537961, 1.9831671237305015, 1.9831275213047679, 1.9831854026497038, 1.9833310603959788, 1.983555753827672, 1.9838516126235661, 1.984204266972864, 1.9846153444229218, 1.985079027240053, 1.9855827938056843, 1.986129935817638, 1.9867088507667374, 1.9873236579358957, 1.9879634999906521, 1.9885396269972921, 1.9891446406345001, 1.9897756643907327, 1.990437391406516, 1.9911194810606403, 1.9917336486786792, 1.9922793746810536, 1.99285701462587, 1.9933771335674881, 1.9939317163857015, 1.9944310742141158, 1.9948734231096907, 1.9952717231951627, 1.9956230776109183, 1.995939444386616, 1.9963105645446688, 1.9966447288031344, 1.9969456172060216, 1.997302800317379, 1.9976244153710299, 1.9979140042106083, 1.9982610129616882, 1.998573466811121, 1.9988548067131309, 1.9991943879503833, 1.999500153912897, 1.9997754719033383, 2.000109630886988, 2.000410514540323, 2.0006814363987, 2.0009253800378217, 2.0011377487199415, 2.001328969869324, 2.0015011493432753, 2.0016561832992257, 2.00178849586555, 2.001907632834135, 2.00201490622228, 2.0021114973974683, 2.002198470087393, 2.0022767820944476, 2.002347295843697, 2.0024107878804895, 2.0024679574223017, 2.002425893871658, 2.0022944787941945, 2.0020826097557833, 1.0145177840018778, 0.9119492438364634, 0.813994923398204, 0.7207455084382421, 0.6322573126538464, 0.548581042061602, 0.46967609643015473, 0.3956784324250726, 0.32644441409708747, 0.2619309656460289, 0.20201304375499962, 0.14665703060322532, 0.09566014205749473, 0.04892604407068468, 0.006375278687247965, -0.03215723986510488, -0.06691369841411499, -0.09802591000720762, -0.12569153560125912, -0.15009583641690405, -0.17139834227414896, -0.1897499803301188, -0.2052793653046181, -0.21818856152520594, -0.228674078309417, -0.23689829028516263, -0.24310094310976949, -0.2474116517503786, -0.24993984099057887, -0.25078403916646863, -0.2502127603914066, -0.24828798315742562, -0.2451590543667037, -0.2409531658448394, -0.23578484373966413, -0.22975720976104652, -0.22305665701461086, -0.2157574775166129, -0.2079265653335236, -0.1997104102413565, -0.1911546576848484, -0.1823866647419603, -0.1734348493807798, -0.16431751651599535, -0.15514468845835117, -0.14592189138240386, -0.1367476413572466, -0.12771064428704815, -0.11879723261344122, -0.11008863997180951, -0.10156447061857765, -0.09329989999214927, -0.08535533439959764, -0.07770619116250721, -0.07031576910739235, -0.06324456192665773, -0.056468065625448816, -0.04994964987858186, -0.04375715538269187, -0.03794438409181344, -0.032473523420639694, -0.027310526136069768, -0.022424735405929486, -0.017874804161476465, -0.013627287737527638, -0.009738328842897825, -0.006172222556250606, -0.002896819428671815, 0.00011682856791783713, 0.0028085298597464448, 0.005210343686484542, 0.007351136849945927, 0.009264184825889421, 0.010878626141938241, 0.01222419585155099, 0.013421208198591827, 0.014398199450468274, 0.015177079162141505, 0.015777575149597434, 0.01621745074597003, 0.01651270042344244, 0.016685009146203855, 0.016832876269498476, 0.016872478695232118, 0.016814597350296223, 0.01666893960402116, 0.01644424617232798, 0.01614838737643387, 0.015795733027136105, 0.015384655577078243, 0.014920972759947082, 0.014417206194315746, 0.013870064182361919, 0.013291149233262578, 0.012676342064104285, 0.012036500009347861, 0.011460373002707858, 0.010855359365499861, 0.010224335609267277, 0.009562608593483901, 0.008880518939359705, 0.008266351321320808, 0.00772062531894635, 0.007142985374130051, 0.006622866432511865, 0.006068283614298453, 0.005568925785884193, 0.0051265768903092646, 0.004728276804837339, 0.004376922389081717, 0.0040605556133839205, 0.0036894354553311803, 0.003355271196865628, 0.003054382793978405, 0.0026971996826210543, 0.00237558462897014, 0.002085995789391726, 0.0017389870383117678, 0.0014265331888789756, 0.0011451932868691284, 0.0008056120496167107, 0.000499846087103073, 0.0002245280966617358, -0.00010963088698812129, -0.000410514540322815, -0.0006814363986999972, -0.0009253800378217214, -0.0011377487199415448, -0.0013289698693239593, -0.0015011493432752765, -0.0016561832992256775, -0.001788495865549855, -0.0019076328341349758, -0.002014906222279844, -0.0021114973974682982, -0.002198470087392934, -0.0022767820944475936, -0.0023472958436969726, -0.00241078788048954, -0.0024679574223016587, -0.002425893871658147, -0.002294478794194532])

In [ ]:
[[0.         0.         0.         0.         1.         0.
  1.         0.         0.305556   1.302778   1.         5.
  0.31381539 0.29703209 0.28242438 0.26921395 0.2581988  0.24863193
  0.2410658  0.23412513 0.22820148 0.22276657 0.21855338 0.21533559
  0.21192642 0.20927844 0.20654308 0.20497225 0.20339007 0.20146078
  0.19977896 0.19898387 0.19824544 0.19753357 0.19752609 0.19772209
  0.19749144 0.19676014 0.19667596 0.19674802 0.19694902 0.19695913
  0.19679149 0.19705153 0.19679285 0.19695753 0.19689724 0.19692835
  0.1970349  0.19690642 0.19715833 0.1971498  0.19690613 0.19704276
  0.19721617 0.19712264 0.19708589 0.197096   0.19714478 0.19722547
  0.19703521 0.1971957  0.19707129 0.19728613 0.19720658 0.19716114
  0.19714386 0.19714982 0.19717495 0.19721591 0.19726994 0.19703744
  0.19714287 0.19725009 0.1970615  0.19720316 0.19734028 0.19717637
  0.19733839 0.19719503 0.19707672 0.19727637 0.19716406 0.19707137
  0.19729218 0.19719733 0.19711905 0.19735174 0.1972667  0.19719652
  0.1971386  0.1970908  0.19734864 0.19728436 0.19723131 0.19718753
  0.1971514  0.19712158 0.19709697 0.19737396 0.19732547 0.19728546
  0.19725244 0.19722518 0.19720269 0.19718413 0.19716881 0.19715617
  0.19714573 0.19713712 0.19713002 0.19712415 0.19711931 0.19711532
  0.19711202 0.1971093  0.19710705 0.1971052  0.19710367 0.19710241
  0.19710137 0.19710051 0.1970998  0.19709921 0.19709873 0.19709833
  0.197098   0.19709773 0.19709751 0.19709732 0.19709717 0.19709704
  0.19709694 0.19709685 0.19709678 0.19709672 0.19709668 0.19709664
  0.1970966  0.19709658 0.19709655 0.19709654 0.19709652 0.19709651
  0.1970965  0.19709649 0.19709648 0.19709648 0.19709647 0.19709647
  0.19709646 0.19709646 0.19709646 0.19709646 0.10670825 0.20045268
  0.2828547  0.35516079 0.41880402 0.474866   0.52454261 0.5684808
  0.60741663 0.64185649 0.67252156 0.70000703 0.72446406 0.74635744
  0.76583179 0.78344761 0.79932744 0.81347129 0.8260814  0.83756356
  0.84801377 0.85751123 0.86636025 0.87467412 0.88234416 0.88924392
  0.89564582 0.90163685 0.9072888  0.91255981 0.91741535 0.9220291
  0.92624113 0.9302227  0.933913   0.93736292 0.94061444 0.94360115
  0.9464704  0.94914162 0.95154832 0.95383781 0.95603056 0.95804239
  0.9599049  0.96164419 0.96328178 0.96483544 0.96621874 0.96756254
  0.96877265 0.96997352 0.97106567 0.9720681  0.97299647 0.97386374
  0.97468057 0.97545579 0.97619665 0.97680807 0.97741376 0.97801471
  0.97851067 0.97902107 0.97954339 0.97997445 0.98043129 0.98080831
  0.98111946 0.98147734 0.98177269 0.98201644 0.9823187  0.98256814
  0.982774   0.98304499 0.98326863 0.9834532  0.98360552 0.98373123
  0.98393607 0.98410512 0.98424463 0.98435977 0.98445479 0.98453321
  0.98459793 0.98475243 0.98487994 0.98498517 0.98507202 0.98514369
  0.98520284 0.98525165 0.98529194 0.98532519 0.98535263 0.98537527
  0.98539396 0.98540938 0.98542211 0.98543262 0.98544129 0.98544844
  0.98545435 0.98545922 0.98546324 0.98546656 0.9854693  0.98547156
  0.98547343 0.98547497 0.98547624 0.98547729 0.98547815 0.98547887
  0.98547945 0.98547994 0.98548034 0.98548067 0.98548095 0.98548117
  0.98548136 0.98548151 0.98548164 0.98548174 0.98548183 0.9854819
  0.98548196 0.98548201 0.98548205 0.98548208 0.98548211 0.98548213
  0.98548215 0.98548217 0.98548218 0.98548219 0.9854822  0.98548221
  0.98548221 0.98548222 1.         0.89329175 0.79954732 0.7171453
  0.64483921 0.58119598 0.525134   0.47545739 0.4315192  0.39258337
  0.35814351 0.32747844 0.29999297 0.27553594 0.25364256 0.23416821
  0.21655239 0.20067256 0.18652871 0.1739186  0.16243644 0.15198623
  0.14248877 0.13363975 0.12532588 0.11765584 0.11075608 0.10435418
  0.09836315 0.0927112  0.08744019 0.08258465 0.0779709  0.07375887
  0.0697773  0.066087   0.06263708 0.05938556 0.05639885 0.0535296
  0.05085838 0.04845168 0.04616219 0.04396944 0.04195761 0.0400951
  0.03835581 0.03671822 0.03516456 0.03378126 0.03243746 0.03122735
  0.03002648 0.02893433 0.0279319  0.02700353 0.02613626 0.02531943
  0.02454421 0.02380335 0.02319193 0.02258624 0.02198529 0.02148933
  0.02097893 0.02045661 0.02002555 0.01956871 0.01919169 0.01888054
  0.01852266 0.01822731 0.01798356 0.0176813  0.01743186 0.017226
  0.01695501 0.01673137 0.0165468  0.01639448 0.01626877 0.01606393
  0.01589488 0.01575537 0.01564023 0.01554521 0.01546679 0.01540207
  0.01524757 0.01512006 0.01501483 0.01492798 0.01485631 0.01479716
  0.01474835 0.01470806 0.01467481 0.01464737 0.01462473 0.01460604
  0.01459062 0.01457789 0.01456738 0.01455871 0.01455156 0.01454565
  0.01454078 0.01453676 0.01453344 0.0145307  0.01452844 0.01452657
  0.01452503 0.01452376 0.01452271 0.01452185 0.01452113 0.01452055
  0.01452006 0.01451966 0.01451933 0.01451905 0.01451883 0.01451864
  0.01451849 0.01451836 0.01451826 0.01451817 0.0145181  0.01451804
  0.01451799 0.01451795 0.01451792 0.01451789 0.01451787 0.01451785
  0.01451783 0.01451782 0.01451781 0.0145178  0.01451779 0.01451779]]

In [158]:
array_simulacion

array([ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        9.51683460e-01,  1.13431010e+00,  9.00000000e+00,  1.20000000e+01,
        1.17864196e+00,  1.13786059e+00,  1.09572523e+00,  1.05259576e+00,
        1.00852745e+00,  9.64492179e-01,  9.18605844e-01,  8.73846122e-01,
        8.29149936e-01,  7.85397156e-01,  7.41620810e-01,  6.99632188e-01,
        6.58310173e-01,  6.17482378e-01,  5.78070837e-01,  5.40913562e-01,
        5.04850776e-01,  4.70597415e-01,  4.38037390e-01,  4.06911057e-01,
        3.77139741e-01,  3.48497487e-01,  3.21780233e-01,  2.96945431e-01,
        2.73644537e-01,  2.52718223e-01,  2.33009107e-01,  2.14312145e-01,
        1.96519965e-01,  1.81458028e-01,  1.66940721e-01,  1.53907230e-01,
        1.42133149e-01,  1.31493673e-01,  1.21876427e-01,  1.14180224e-01,
        1.07222170e-01,  1.00929368e-01,  9.61583167e-02,  9.18431390e-02,
        8.88611842e-02,  

In [159]:
len(array_simulacion)

438

In [160]:
array_simulacion.shape

(438,)

In [161]:
array_simulacion = np.array(array_simulacion).reshape(-1,1)

In [162]:
array_simulacion.shape

(438, 1)

In [163]:
array_simulacion

array([[ 1.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 1.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 9.51683460e-01],
       [ 1.13431010e+00],
       [ 9.00000000e+00],
       [ 1.20000000e+01],
       [ 1.17864196e+00],
       [ 1.13786059e+00],
       [ 1.09572523e+00],
       [ 1.05259576e+00],
       [ 1.00852745e+00],
       [ 9.64492179e-01],
       [ 9.18605844e-01],
       [ 8.73846122e-01],
       [ 8.29149936e-01],
       [ 7.85397156e-01],
       [ 7.41620810e-01],
       [ 6.99632188e-01],
       [ 6.58310173e-01],
       [ 6.17482378e-01],
       [ 5.78070837e-01],
       [ 5.40913562e-01],
       [ 5.04850776e-01],
       [ 4.70597415e-01],
       [ 4.38037390e-01],
       [ 4.06911057e-01],
       [ 3.77139741e-01],
       [ 3.48497487e-01],
       [ 3.21780233e-01],
       [ 2.96945431e-01],
       [ 2.73644537e-01],
       [ 2.52718223e-01],
       [ 2.3

In [164]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
array_simulacion_scalado = scaler.fit_transform(array_simulacion)

In [165]:
len(array_simulacion_scalado)

438

In [167]:
type(array_simulacion_scalado)

numpy.ndarray

In [166]:
array_simulacion_scalado

array([[ 1.88803239e-01],
       [-6.97153147e-01],
       [-6.97153147e-01],
       [-6.97153147e-01],
       [ 1.88803239e-01],
       [-6.97153147e-01],
       [-6.97153147e-01],
       [-6.97153147e-01],
       [ 1.45996892e-01],
       [ 3.07796130e-01],
       [ 7.27645433e+00],
       [ 9.93432348e+00],
       [ 3.47072221e-01],
       [ 3.10941712e-01],
       [ 2.73611616e-01],
       [ 2.35400788e-01],
       [ 1.96358184e-01],
       [ 1.57344858e-01],
       [ 1.16691567e-01],
       [ 7.70364055e-02],
       [ 3.74375340e-02],
       [-1.32552067e-03],
       [-4.01094539e-02],
       [-7.73095422e-02],
       [-1.13919045e-01],
       [-1.50090691e-01],
       [-1.85007597e-01],
       [-2.17927322e-01],
       [-2.49877378e-01],
       [-2.80224362e-01],
       [-3.09071124e-01],
       [-3.36647698e-01],
       [-3.63023785e-01],
       [-3.88399572e-01],
       [-4.12069894e-01],
       [-4.34072446e-01],
       [-4.54716022e-01],
       [-4.73255823e-01],
       [-4.9

In [169]:
array_simulacion_scalado = np.array(array_simulacion_scalado).reshape(1,-1)
array_simulacion_scalado

array([[ 1.88803239e-01, -6.97153147e-01, -6.97153147e-01,
        -6.97153147e-01,  1.88803239e-01, -6.97153147e-01,
        -6.97153147e-01, -6.97153147e-01,  1.45996892e-01,
         3.07796130e-01,  7.27645433e+00,  9.93432348e+00,
         3.47072221e-01,  3.10941712e-01,  2.73611616e-01,
         2.35400788e-01,  1.96358184e-01,  1.57344858e-01,
         1.16691567e-01,  7.70364055e-02,  3.74375340e-02,
        -1.32552067e-03, -4.01094539e-02, -7.73095422e-02,
        -1.13919045e-01, -1.50090691e-01, -1.85007597e-01,
        -2.17927322e-01, -2.49877378e-01, -2.80224362e-01,
        -3.09071124e-01, -3.36647698e-01, -3.63023785e-01,
        -3.88399572e-01, -4.12069894e-01, -4.34072446e-01,
        -4.54716022e-01, -4.73255823e-01, -4.90717241e-01,
        -5.07281934e-01, -5.23045029e-01, -5.36389248e-01,
        -5.49250949e-01, -5.60798053e-01, -5.71229376e-01,
        -5.80655487e-01, -5.89175948e-01, -5.95994448e-01,
        -6.02158981e-01, -6.07734129e-01, -6.11961072e-0

In [170]:
y_pred = loaded_model.predict(array_simulacion_scalado)
y_pred

[array([[0.77521753]], dtype=float32), array([[1.2059395]], dtype=float32)]

In [173]:
y_pred[0][0][0]

0.77521753

In [177]:
y_pred[1][0][0]

1.2059395